In [1]:
# This file calculates Line coverage by clones in each of the files detected by Detectors

In [2]:
import pandas as pd
import mysql.connector
from mysql.connector import Error

In [3]:
data_loc ="C:/Users/MD NADIM/Documents/NetBeansProjects/FindCochangeByClone/data_files/final_dataset/"

In [45]:
# List of Subject Systems and Clone Detectors!!!
subject_systems=["brlcad", "carol", "ctags", "freecol", "jabref", "jedit"]
clone_detectors=["conqat", "deckard_2_0", "iclones", "nicad5", "simcad", "simian"]

# This is the declaration if you want to check the functinality for one subject sytems and clone detector. 
# subject_systems=["brlcad"]
# clone_detectors=["conqat"]
record=[['Subject System', 'Clone Detector', 'MaxV', 'MaxV-1', 'MaxV-2', 'MaxV-3', 'MaxV-4']]
line=[]

# Work for every subject systems. 
for ss in subject_systems: 
    try:
        mySQLconnection = mysql.connector.connect(host='localhost',
                                 database='cochange_'+ss,
                                 #database='vua',                 
                                 user='root',
                                 password='')        

        for cd in clone_detectors:
            line.append(ss)
            line.append(cd)
            for i in range(5):
                cursor1 = mySQLconnection.cursor()
                query="SELECT * FROM `clones_"+cd+"` WHERE `version`= (SELECT DISTINCT `version` FROM `clones_"+cd+"` ORDER BY `version`  DESC LIMIT "+str(i)+", 1) ORDER BY `version`, `file_name`, `startline`, `endline`"
                #print(query)
                cursor1.execute(query)
                records = cursor1.fetchall()
                line.append(calculate_lines(records))
                #print(len(records))
            #print(line)
            record.append(line)
            line=[]


    except Error as e:
        print("Error reading data from MySQL table", e, ss)
    finally:
        if (mySQLconnection.is_connected()):
            mySQLconnection.close()
            cursor1.close()
            #print("MySQL connection is closed")
pd.DataFrame(record).to_csv("generated_csv/line_coverage.csv", header=False, index=False)

In [8]:
from IPython.display import clear_output
# This will find the number of line coverage in all revisions. 
# List of Subject Systems and Clone Detectors!!!
subject_systems=["brlcad", "carol", "ctags", "freecol", "jabref", "jedit"]
clone_detectors=["conqat", "deckard_2_0", "iclones", "nicad5", "simcad", "simian"]

# This is the declaration if you want to check the functinality for one subject sytems and clone detector. 
# subject_systems=["brlcad"]
# clone_detectors=["conqat"]
record=[]
line=[]

# Work for every subject systems. 
for ss in subject_systems: 
    try:
        mySQLconnection = mysql.connector.connect(host='localhost',
                                 database='cochange_'+ss,
                                 #database='vua',                 
                                 user='root',
                                 password='')        

        for cd in clone_detectors:
            line.append(ss)
            line.append(cd)
            cursor_num = mySQLconnection.cursor()
            cursor_num.execute("SELECT DISTINCT `version` FROM `clones_"+cd+"`")
            versions = cursor_num.fetchall()
            for v in versions:
#                 if(v[0]==20):
#                     break
                clear_output(wait=True)
                print("Working:",ss,cd, v[0])
                cursor1 = mySQLconnection.cursor()
                query="SELECT * FROM `clones_"+cd+"` WHERE `version`='"+str(v[0])+"'  ORDER BY `version`, `file_name`, `startline`, `endline`"
#                 print(query)
#                 break
                cursor1.execute(query)
                records = cursor1.fetchall()
                line.append(calculate_lines(records))
                #print(len(records))
            #print(line)
            record.append(line)
            line=[]
#             break


    except Error as e:
        print("Error reading data from MySQL table", e, ss)
    finally:
        if (mySQLconnection.is_connected()):
            mySQLconnection.close()
            cursor1.close()
            #print("MySQL connection is closed")
pd.DataFrame(record).to_csv("generated_csv/line_coverage_all.csv", header=False, index=False)

Working: jedit simian 4001


In [3]:
def calculate_lines(records):
    num_lines=0
    total=0
    store_lines=[]

    for i in range(len(records)-1):
    #     cur_rev=records[i][1]     #Revision
    #     cur_filename=records[i][3]     #Filename
    #     cur_sline=records[i][4]     #Startline
    #     cur_eline=records[i][5]     #Endline
        for ln in range(records[i][4], records[i][5]+1):
            if(ln not in store_lines):
                store_lines.append(ln)

        if(records[i][3] !=records[i+1][3]):
            #print(records[i][3], len(store_lines))
            total+=len(store_lines)
            store_lines=[]

    #Here is the last file :) 
    #print(records[i-1][3], len(store_lines)) #To show the lines in a file
    total+=len(store_lines)      

    return total

In [16]:
# List of Subject Systems and Clone Detectors!!!
subject_systems=["brlcad", "carol", "ctags", "freecol", "jabref", "jedit"]
clone_detectors=["conqat", "deckard", "iclones", "nicad5", "simcad", "simian"]

# This is the declaration if you want to check the functinality for one subject sytems and clone detector. 
# subject_systems=["brlcad"]
# clone_detectors=["conqat"]


# Work for every subject systems. 
for ss in subject_systems: 
    try:
        mySQLconnection = mysql.connector.connect(host='localhost',
                                 database='cochange_'+ss,
                                 #database='vua',                 
                                 user='root',
                                 password='')        

        cursor1 = mySQLconnection.cursor()

        # Read the cochange file stored in CSV. 
        df_cochange = pd.read_csv(data_loc+"all_performance_files/"+ss+"_performance.csv", sep=',', header=0, encoding='utf-8')
        result_set = [["change_id", "num_all_cochange", "num_cloned_cochange"]]

        print(ss)
        for i in range(len(df_cochange)):
            query="SELECT COUNT(`id`)-1 total_cc FROM `change_info`WHERE `version`=(SELECT `version` FROM `change_info` WHERE `id`='"+str(df_cochange["change_id"][i])+"')"
            cursor1.execute(query)
            records = cursor1.fetchall()
            #if(records[0][0]<1):   #For debugging, if any calculation is falling under 1 or not, which is not acceptable
#             print(query)
#             print(records[0][0])
#             print(df_cochange["change_id"][i])
#             print(df_cochange["num_change"][i]) 
#             print()
            
            result_set.append([df_cochange["change_id"][i], records[0][0], df_cochange["num_change"][i]])
        
        pd.DataFrame(result_set).to_csv(data_loc+"num_cloned_cochnage/"+ss+"_cloned_cochnage.csv", header=False, index=False)


    except Error as e:
        print("Error reading data from MySQL table", e, ss)
    finally:
        if (mySQLconnection.is_connected()):
            mySQLconnection.close()
            cursor1.close()
            #print("MySQL connection is closed")

brlcad
carol
ctags
freecol
jabref
jedit


In [18]:
#This if for test!!!
print(df_cochange.shape)
print(df_cochange["conqat"][0])
print(len(df_cochange))

(2103, 7)
42252
2103
